<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5c8274f1f19015b952cacccf4c0ddce6de7135ae77d1ee00a15786a8b6884c05
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-18 11:19:09
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: -72.20 K (-0.5%)
Current PnL: -20.81 L (-13.64%)
CY Booked + Current PnL: -6.85 L (-4.49%)
-------------------
Total profit:  1.51 L
Total loss:  -22.32 L
-------------------
Total Booked + Current PnL: 20.13 L (16.03%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.62%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.37 C
Est FTT PnL: 91.64 L (63.18%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.29%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,151.67,86.0,M-SC,0.71,165081.0,-9478.0,9509.0,0.76,-5.43,5.76,0.02,234.0,-1.00,1.14,43.59,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.64,49.0,M-SC,3.60,86460.0,-14317.0,14413.0,-0.65,-14.21,16.67,0.09,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-14.73,50.0,X-MC,3.67,89025.0,-1653.0,16559.0,0.39,-1.82,18.60,16.44,63.0,-0.10,0.61,17.98,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.07,57.0,X-MC,7.55,188864.0,14544.0,20662.0,-0.59,8.34,10.94,20.19,68.0,0.70,1.30,24.72,X40,ATH,INSURANCE
84,VOLTAS,1530.00,-2.18,53.0,X-MC,3.67,207690.0,15948.0,21807.0,0.78,8.32,10.50,19.69,78.0,0.73,1.43,15.68,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,117.55,72.0,H-SC,16.21,148013.0,-70483.0,98680.0,12.88,-32.26,66.67,12.90,161.0,-0.71,1.02,23.07,OX40N,NTT,FINANCE
64,SATIN,274.00,-22.47,64.0,H-SC,2.45,239908.0,-36376.0,179595.0,2.49,-13.17,74.86,51.83,146.0,-0.20,1.65,16.87,XY24,NTT,FINANCE
12,BERGEPAINT,680.00,-2.76,73.0,X-MC,2.62,237735.0,10396.0,37657.0,1.54,4.57,15.84,21.14,74.0,0.28,1.64,34.00,XY24,NTT,PAINTS
16,CERA,9475.00,-28.16,33.0,H-SC,2.83,130996.0,-44907.0,86929.0,1.33,-25.53,66.36,23.89,157.0,-0.52,0.90,13.13,OX40N,NTT,CERAMICS
29,HAPPSTMNDS,1480.71,-24.87,47.0,H-SC,14.57,80976.0,-47189.0,155935.0,0.78,-36.82,192.57,84.85,147.0,-0.30,0.56,3.93,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SURYODAY,240.00,55.70,46.0,H-SC,13.19,142902.0,-36169.0,101903.0,-3.04,-20.20,71.31,36.71,167.0,-0.35,0.99,41.79,XR,NTT,BANKS
26,GILLETTE,11206.78,-18.88,29.0,X-SC,5.95,250575.0,-3321.0,85621.0,-2.73,-1.31,34.17,32.42,85.0,-0.04,1.73,13.59,X40,ATH,FMCG
13,BSOFT,831.70,-11.48,55.0,H-SC,8.32,97028.0,-43625.0,115056.0,-2.65,-31.02,118.58,50.79,151.0,-0.38,0.67,12.97,XR,ATH,IT
85,WHIRLPOOL,2270.00,-43.39,35.0,M-SC,8.43,111026.0,-3772.0,97814.0,-2.33,-3.29,88.10,81.92,236.0,-0.04,0.77,31.44,XR,NTT,DURABLES
14,CAMPUS,393.00,-26.26,42.0,M-SC,3.46,153573.0,-14202.0,70045.0,-2.09,-8.47,45.61,33.28,222.0,-0.20,1.06,19.03,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.11,146795.0,-849.0,160168.0,-1.03,-0.58,109.11,107.91,216.0,-0.01,1.01,2.41,OX40N,ATH,CABLES
70,STARHEALTH,761.00,36.29,67.0,H-SC,15.65,280976.0,-147.0,133014.0,-0.72,-0.05,47.34,47.26,171.0,-0.00,1.94,50.54,XY24,NTT,INSURANCE
86,WIPRO,420.00,-14.70,48.0,M-LC,5.57,149941.0,-1004.0,110462.0,-0.91,-0.67,73.67,72.51,99.0,-0.01,1.03,5.32,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.11,146795.0,-849.0,160168.0,-1.03,-0.58,109.11,107.91,216.0,-0.01,1.01,2.41,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,151.67,86.0,M-SC,0.71,165081.0,-9478.0,9509.0,0.76,-5.43,5.76,0.02,234.0,-1.00,1.14,43.59,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.64,49.0,M-SC,3.60,86460.0,-14317.0,14413.0,-0.65,-14.21,16.67,0.09,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.17,48.0,H-SC,2.23,222471.0,-47196.0,83538.0,-0.20,-17.50,37.55,13.47,158.0,-0.56,1.53,13.12,XY24,NTT,PAINTS
44,JCHAC,2282.00,19986.96,65.0,M-SC,1.45,98123.0,-29682.0,29672.0,0.09,-23.22,30.24,-0.01,233.0,-1.00,0.68,11.11,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,21.85,60.0,M-SC,36.46,164358.0,15138.0,129037.0,-0.57,10.14,78.51,96.62,245.0,0.12,1.13,77.87,XY25,NTT,FINANCE
38,INDIAMART,4810.62,-47.96,49.0,H-SC,4.57,129442.0,6106.0,125520.0,-1.34,4.95,96.97,106.72,139.0,0.05,0.89,29.49,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.96,49.0,H-SC,4.57,129442.0,6106.0,125520.0,-1.34,4.95,96.97,106.72,139.0,0.05,0.89,29.49,AR,ATH,MISC
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.11,146795.0,-849.0,160168.0,-1.03,-0.58,109.11,107.91,216.0,-0.01,1.01,2.41,OX40N,ATH,CABLES
86,WIPRO,420.00,-14.70,48.0,M-LC,5.57,149941.0,-1004.0,110462.0,-0.91,-0.67,73.67,72.51,99.0,-0.01,1.03,5.32,XR,NTT,IT
87,ZYDUSLIFE,1286.17,-15.41,34.0,H-MC,5.41,206105.0,-3469.0,78134.0,-0.23,-1.66,37.91,35.63,119.0,-0.04,1.42,14.74,AR,ATH,PHARMA
85,WHIRLPOOL,2270.00,-43.39,35.0,M-SC,8.43,111026.0,-3772.0,97814.0,-2.33,-3.29,88.10,81.92,236.0,-0.04,0.77,31.44,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.89,24.0,X-SC,10.35,83847.0,-44823.0,90119.0,-0.43,-34.84,107.48,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
77,TMPV,1065.00,-20.63,25.0,X-LC,0.20,242939.0,-33471.0,454636.0,-0.48,-12.11,187.14,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
26,GILLETTE,11206.78,-18.88,29.0,X-SC,5.95,250575.0,-3321.0,85621.0,-2.73,-1.31,34.17,32.42,85.0,-0.04,1.73,13.59,X40,ATH,FMCG
23,DMART,5391.45,-17.88,29.0,X-LC,5.23,200220.0,-10187.0,69356.0,-0.79,-4.84,34.64,28.12,19.0,-0.15,1.38,17.64,X40N,ATH,FMCG
34,HONAUT,58357.33,-26.34,29.0,X-SC,1.14,104805.0,-23133.0,70272.0,-1.45,-18.08,67.05,36.84,90.0,-0.33,0.72,7.29,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-20.63,25.0,X-LC,0.20,242939.0,-33471.0,454636.0,-0.48,-12.11,187.14,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,44.0,X-LC,0.90,198118.0,-2020.0,22902.0,-0.48,-1.01,11.56,10.44,5.0,-0.09,1.37,4.50,X40,NTT,FMCG
34,HONAUT,58357.33,-26.34,29.0,X-SC,1.14,104805.0,-23133.0,70272.0,-1.45,-18.08,67.05,36.84,90.0,-0.33,0.72,7.29,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.69,62.0,X-MC,1.33,183359.0,6621.0,48113.0,-0.76,3.75,26.24,30.97,75.0,0.14,1.26,16.78,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-1.39,60.0,H-LC,1.66,160750.0,-25345.0,72820.0,-0.54,-13.62,45.30,25.51,49.0,-0.35,1.11,18.46,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.89,24.0,X-SC,10.35,83847.0,-44823.0,90119.0,-0.43,-34.84,107.48,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51769.93,-30.18,32.0,X-MC,7.97,155300.0,-8440.0,51777.0,-1.63,-5.15,33.34,26.47,55.0,-0.16,1.07,0.00,X40,ATH,APPARELS
77,TMPV,1065.00,-20.63,25.0,X-LC,0.20,242939.0,-33471.0,454636.0,-0.48,-12.11,187.14,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
56,QUESS,986.00,-53.17,33.0,X-SC,37.18,46673.0,-18333.0,169260.0,-2.08,-28.20,362.65,232.18,83.0,-0.11,0.32,0.60,XY24,NTT,MISC
9,BAJAJHFL,181.50,-19.01,33.0,X-MC,6.72,175103.0,-25403.0,122555.0,-0.58,-12.67,69.99,48.45,64.0,-0.21,1.21,0.97,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,986.00,-53.17,33.0,X-SC,37.18,46673.0,-18333.0,169260.0,-2.08,-28.20,362.65,232.18,83.0,-0.11,0.32,0.60,XY24,NTT,MISC
60,RELAXO,1176.00,-41.34,39.0,X-SC,3.02,79994.0,-65166.0,141093.0,-0.56,-44.89,176.38,52.31,91.0,-0.46,0.55,7.08,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,9.89,24.0,X-SC,10.35,83847.0,-44823.0,90119.0,-0.43,-34.84,107.48,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-14.73,50.0,X-MC,3.67,89025.0,-1653.0,16559.0,0.39,-1.82,18.60,16.44,63.0,-0.10,0.61,17.98,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.34,29.0,X-SC,1.14,104805.0,-23133.0,70272.0,-1.45,-18.08,67.05,36.84,90.0,-0.33,0.72,7.29,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-26.86,57.0,X-LC,8.34,290639.0,-55317.0,122010.0,-0.33,-15.99,41.98,19.28,1.0,-0.45,2.00,7.44,X40,ATH,IT
41,INFY,2275.00,-21.03,50.0,X-LC,3.85,317647.0,4759.0,166923.0,-1.08,1.52,52.55,54.87,2.0,0.03,2.19,9.93,X40,BTT,IT
77,TMPV,1065.00,-20.63,25.0,X-LC,0.20,242939.0,-33471.0,454636.0,-0.48,-12.11,187.14,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
83,VBL,671.64,-18.89,46.0,X-LC,6.04,290960.0,-24882.0,137537.0,-1.22,-7.88,47.27,35.67,4.0,-0.18,2.01,4.88,X40N,ATH,FMCG
43,ITC,452.00,-40.74,44.0,X-LC,0.90,198118.0,-2020.0,22902.0,-0.48,-1.01,11.56,10.44,5.0,-0.09,1.37,4.50,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7705.56,67.0,L-SC,17.71,85625.0,-8185.0,84803.0,-0.14,-8.73,99.04,81.67,271.0,-0.10,0.59,65.82,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.70,50.0,H-SC,7.10,92190.0,-5790.0,27390.0,-1.68,-5.91,29.71,22.05,164.0,-0.21,0.64,34.00,XR,ATH,FINANCE
71,SURYODAY,240.00,55.70,46.0,H-SC,13.19,142902.0,-36169.0,101903.0,-3.04,-20.20,71.31,36.71,167.0,-0.35,0.99,41.79,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,69.70,56.0,H-SC,10.26,146978.0,-35797.0,146860.0,-1.83,-19.59,99.92,60.77,159.0,-0.24,1.01,32.36,XR,NTT,JEWELLERY
66,SHALBY,327.00,1032.38,34.0,M-SC,17.00,154089.0,-27054.0,72530.0,-0.39,-14.94,47.07,25.10,250.0,-0.37,1.06,23.49,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,151.67,86.0,M-SC,0.71,165081.0,-9478.0,9509.0,0.76,-5.43,5.76,0.02,234.0,-1.00,1.14,43.59,OX40N,NTT,PAINTS
81,VAIBHAVGBL,521.00,69.70,56.0,H-SC,10.26,146978.0,-35797.0,146860.0,-1.83,-19.59,99.92,60.77,159.0,-0.24,1.01,32.36,XR,NTT,JEWELLERY
5,ASIANPAINT,3460.26,0.21,83.0,X-LC,10.14,254848.0,3080.0,49644.0,0.44,1.22,19.48,20.95,31.0,0.06,1.76,37.30,X40,ATH,PAINTS
0,5PAISA,593.00,117.55,72.0,H-SC,16.21,148013.0,-70483.0,98680.0,12.88,-32.26,66.67,12.90,161.0,-0.71,1.02,23.07,OX40N,NTT,FINANCE
13,BSOFT,831.70,-11.48,55.0,H-SC,8.32,97028.0,-43625.0,115056.0,-2.65,-31.02,118.58,50.79,151.0,-0.38,0.67,12.97,XR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,44.28
2,50,75.05


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.80
MC    28.60
LC    24.56
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.02
X40      20.44
X40N     11.51
XY25     10.11
XR        9.49
AR        9.03
OX40N     7.75
X200      1.79
SR        1.01
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.41
X-MC    22.24
X-LC    19.64
M-SC    13.27
X-SC     7.71
H-MC     4.67
H-LC     2.90
L-SC     1.41
M-MC     1.36
M-LC     1.03
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.55,-4.90,38.83
IT,13.29,-15.48,75.14
FINANCE,10.63,-10.20,62.02
MISC,7.43,-20.33,81.03
PAINTS,6.07,-4.91,20.49
ELECTRICAL,5.63,-10.78,50.05
INSURANCE,4.50,3.22,30.89
PHARMA,3.40,-1.07,34.97
AUTO,3.35,-18.40,131.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3375421.0,21
XR,1278458.0,13
AR,1263534.0,10
X40,1004864.0,14
X40N,701736.0,9
OX40N,666632.0,10
XY25,467781.0,7
SR,274855.0,2
MH,66519.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3461204.0,25
M-SC,1427329.0,16
X-LC,1220905.0,11
X-MC,1166715.0,16
X-SC,844980.0,8
H-MC,401238.0,3
L-SC,256001.0,3
H-LC,136682.0,2
M-LC,110462.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1164274.0      6
           AR         847684.0      5
           XR         798697.0      7
M-SC       XY24       755903.0      6
X-LC       XY24       522115.0      2
X-MC       X40        416142.0      7
X-LC       X40        412982.0      5
X-SC       XY24       403134.0      3
X-MC       XY24       346005.0      3
H-SC       OX40N      309175.0      4
           SR         274855.0      2
M-SC       OX40N      270185.0      5
X-SC       X40N       266106.0      3
X-MC       X40N       228737.0      4
H-MC       AR         217248.0      2
X-LC       X40N       206893.0      2
H-MC       XY24       183990.0      1
X-MC       XY25       175831.0      2
X-SC       X40        175740.0      2
L-SC       XR         168729.0      2
M-SC       XR         146422.0      2
           XY25       129037.0      1
           AR         125782.0      2
M-LC       XR         110462.0      1
L-SC       OX40N       87272.0      1
X-LC       XY25        78915.0      2
H-LC       AR          72820.0      1
H-SC       MH          66519.0      1
H-LC       X200        63862.0      1
L-MC       XR          54148.0      1
M-MC       XY25        47527.0      1
L-LC       XY25        36471.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
